In [13]:
import numpy as np
import pytorch as 

In [21]:
def phi(t, x, p , theta) :
    return 0

def argmaxH(t,x,p,q):
    return 0

def b(t, u ,x):
    return -x/4 + u

def sigma(t, u, x):
    return x/5 + u

def h(x):
    return (np.dot(Q@x, x))/2
    
def f(t, x, u):
    return (np.dot(x,x)/2 + np.dot(2*u , u))/2
    
def Hx(t, x , u , p , q):
    return -p/4 + q/5 -x/2 



In [30]:
max_steps = 100
n = 25
M = 64
T = 0.1
Q = np.eye(n)
p0 = np.array([-0.9586 for _ in range(n+1)])

In [31]:
t = np.arange(0,T,n)
x = np.array([[0 for _ in range(n+1)] for _ in range(M)])
p = np.array([[0 for _ in range(n+1)] for _ in range(M)])
q = np.array([0 for _ in range(n)])
u = np.array([0 for _ in range(n)])
p[0] = [p0 for _ in range(M)]
for l in range(max_steps):
    x[0] = [x0 for _ in range(M)]
    
    for i in range(n):
        q[i] = phi(t[i] , x[i] , p[i] , theta)
        u[i] = argmaxH(t[i], x[i] , p[i] , q[i])
        for o in range(M):
            x[i+1][o] = x[i] + b(t[i], x[i], u[i])*step + sigma(t[i], x[i], u[i])*sqrt(step)*random.normal()
            p[i+1][o] = p[i] - Hx(t[i], x[i], u[i], p[i], q[i])*step + q[i]*sqrt(step)*random.normal()
    J=0
    for k in range(M):
        J+=np.sum(f(t,x[k], u))*T/n + h(x[k][-1])
    J/= M
    loss = np.sum((np.square(h_x(x[:,-1]) + p[:,-1])))/M
    theta , p[0] = theta - nu * gradient_loss

#returns 4-tuple processes (x, u, p , q)_t
        
        
    
    
    

ValueError: setting an array element with a sequence. The requested array would exceed the maximum number of dimension of 1.

In [29]:
p0

array([-0.9586, -0.9586, -0.9586, -0.9586, -0.9586, -0.9586, -0.9586,
       -0.9586, -0.9586, -0.9586, -0.9586, -0.9586, -0.9586, -0.9586,
       -0.9586, -0.9586, -0.9586, -0.9586, -0.9586, -0.9586, -0.9586,
       -0.9586, -0.9586, -0.9586, -0.9586])